In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve, auc, f1_score, accuracy_score, precision_recall_curve)
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Input, Attention, LayerNormalization
from tensorflow.keras.optimizers import Adam
import optuna
import shap
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2025-10-15 15:30:15.343064: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 15:30:15.350818: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-15 15:30:15.372643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760522415.409790   25484 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760522415.420782   25484 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760522415.450671   25484 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
class MilitaryRoboticMaintenance:
    def __init__(self):
        self.rf_model = None
        self.attention_gru_model = None
        self.ensemble_model = None
        self.scaler = StandardScaler()
        self.feature_names = None
        self.best_params = None
        
    def generate_synthetic_data(self, n_samples=1000):
        np.random.seed(42)
        
        data = {
            'system_temperature': np.random.normal(65, 15, n_samples),
            'power_unit_temperature': np.random.normal(75, 12, n_samples),
            'ambient_temperature': np.random.normal(25, 5, n_samples),
            'system_vibration': np.random.exponential(1.5, n_samples),
            'hydraulic_pressure': np.random.normal(2000, 400, n_samples),
            'current_draw': np.random.normal(50, 15, n_samples),
            'operational_hours': np.random.uniform(0, 5000, n_samples),
            'previous_failures': np.random.poisson(1.5, n_samples),
        }
        
        failure_probability = (
            0.3 * (data['system_temperature'] > 70).astype(int) +
            0.4 * (data['power_unit_temperature'] > 80).astype(int) +
            0.4 * (data['system_vibration'] > 2.0).astype(int) +
            0.2 * (data['operational_hours'] > 4000).astype(int) +
            0.3 * (data['previous_failures'] > 2).astype(int)
        ) / 1.6
        
        data['is_failure'] = np.random.binomial(1, failure_probability)
        
        # Add RUL (Remaining Useful Life) estimation
        max_life = 5000
        data['rul'] = max_life - data['operational_hours'] - \
                      data['previous_failures'] * 200 - \
                      np.maximum(0, data['system_temperature'] - 60) * 10
        data['rul'] = np.maximum(0, data['rul'])
        
        df = pd.DataFrame(data)
        df['timestamp'] = pd.date_range(start='2024-01-01', periods=n_samples, freq='h')
        
        return df
    
    def preprocess_data(self, data):
        """Preprocess and split data"""
        self.feature_names = [col for col in data.columns 
                             if col not in ['is_failure', 'timestamp', 'rul']]
        X = data[self.feature_names]
        y = data['is_failure']
        y_rul = data['rul']
        
        X_scaled = self.scaler.fit_transform(X)
        
        return train_test_split(X_scaled, y, y_rul, test_size=0.3, random_state=42)
    
    def optimize_rf_with_optuna(self, X_train, y_train, n_trials=50):
        print("\n Starting Bayesian Optimization with Optuna...")
        
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 300),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'random_state': 42
            }
            
            model = RandomForestClassifier(**params)
            score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1', n_jobs=-1).mean()
            return score
        
        study = optuna.create_study(direction='maximize', study_name='RF_optimization')
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        
        self.best_params = study.best_params
        print(f"\n Best Parameters: {self.best_params}")
        print(f"Best F1 Score: {study.best_value:.4f}")
        
        return self.best_params
    
    def train_optimized_rf(self, X_train, y_train):
        if self.best_params is None:
            print("Using default parameters. Run optimize_rf_with_optuna first.")
            self.best_params = {
                'n_estimators': 100,
                'max_depth': 10,
                'min_samples_split': 5,
                'min_samples_leaf': 2,
                'max_features': 'sqrt',
                'random_state': 42
            }
        
        self.rf_model = RandomForestClassifier(**self.best_params)
        self.rf_model.fit(X_train, y_train)
        print("Random Forest trained successfully!")
    
    def build_attention_gru(self, input_shape):
        """Build GRU with Attention mechanism"""
        inputs = Input(shape=input_shape)
        
        gru_out = GRU(64, return_sequences=True)(inputs)
        gru_out = Dropout(0.3)(gru_out)
        
        # Attention mechanism
        attention = Attention()([gru_out, gru_out])
        attention = LayerNormalization()(attention)
        
        attention_flat = tf.keras.layers.GlobalAveragePooling1D()(attention)
        dense = Dense(32, activation='relu')(attention_flat)
        dense = Dropout(0.3)(dense)
        outputs = Dense(1, activation='sigmoid')(dense)
        
        model = Model(inputs=inputs, outputs=outputs)
        return model
    
    def train_deep_models(self, X_train, X_test, y_train, y_test, epochs=30):
        print("\nTraining Deep Learning Models...")
        
        # Reshape for sequential models
        X_train_seq = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test_seq = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    
        
        # Attention-GRU
        print("2️⃣ Training Attention-GRU...")
        self.attention_gru_model = self.build_attention_gru((1, X_train.shape[1]))
        self.attention_gru_model.compile(optimizer=Adam(learning_rate=0.001),
                                        loss='binary_crossentropy',
                                        metrics=['accuracy'])
        
        history_att = self.attention_gru_model.fit(X_train_seq, y_train,
                                                   epochs=epochs, batch_size=32,
                                                   validation_split=0.2, verbose=0)
        
        print("Deep learning models trained successfully!")
        return history_att, X_test_seq
    
    def evaluate_all_models(self, X_test, X_test_seq, y_test, n_runs=5):
        print("\nCOMPREHENSIVE MODEL EVALUATION")
        print("="*70)
        
        results = {
            'Random Forest': {'acc': [], 'f1': [], 'prec': [], 'rec': []},
            'GRU': {'acc': [], 'f1': [], 'prec': [], 'rec': []},
            'Attention-GRU': {'acc': [], 'f1': [], 'prec': [], 'rec': []}
        }
        
        # Multiple runs with different seeds
        for run in range(n_runs):
            # Random Forest
            rf_pred = self.rf_model.predict(X_test)
            results['Random Forest']['acc'].append(accuracy_score(y_test, rf_pred))
            results['Random Forest']['f1'].append(f1_score(y_test, rf_pred))
            
            # Attention-GRU
            att_pred_prob = self.attention_gru_model.predict(X_test_seq, verbose=0)
            att_pred = (att_pred_prob > 0.5).astype(int).flatten()
            results['Attention-GRU']['acc'].append(accuracy_score(y_test, att_pred))
            results['Attention-GRU']['f1'].append(f1_score(y_test, att_pred))
        
        # Print results with standard deviations
        for model_name, metrics in results.items():
            acc_mean = np.mean(metrics['acc'])
            acc_std = np.std(metrics['acc'])
            f1_mean = np.mean(metrics['f1'])
            f1_std = np.std(metrics['f1'])
            
            print(f"\n{model_name}:")
            print(f"  Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
            print(f"  F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
        
        print("="*70)
        return results
    
    def create_ensemble(self, X_train, y_train):
        """Create soft voting ensemble"""
        print("\n Creating Soft Voting Ensemble...")
        
        # Note: For simplicity, using RF only in sklearn ensemble
        # In practice, you'd need custom estimators for deep learning models
        self.ensemble_model = self.rf_model  # Simplified for demonstration
        print(" Ensemble model created!")
    
    def plot_pr_curves(self, X_test, X_test_seq, y_test):
        """Plot Precision-Recall curves for all models"""
        print("\n Generating PR Curves...")
        
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        
        # Random Forest
        rf_proba = self.rf_model.predict_proba(X_test)[:, 1]
        precision, recall, _ = precision_recall_curve(y_test, rf_proba)
        axes[0].plot(recall, precision, 'b-', linewidth=2)
        axes[0].set_xlabel('Recall')
        axes[0].set_ylabel('Precision')
        axes[0].set_title('Random Forest - PR Curve')
        axes[0].grid(True, alpha=0.3)
        
        # Attention-GRU
        att_proba = self.attention_gru_model.predict(X_test_seq, verbose=0).flatten()
        precision, recall, _ = precision_recall_curve(y_test, att_proba)
        axes[2].plot(recall, precision, 'r-', linewidth=2)
        axes[2].set_xlabel('Recall')
        axes[2].set_ylabel('Precision')
        axes[2].set_title('Attention-GRU - PR Curve')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def estimate_rul(self, X_test, y_rul_test):
        """Remaining Useful Life estimation"""
        print("\n REMAINING USEFUL LIFE (RUL) ESTIMATION")
        print("="*70)
        
        # Build RUL regression model
        rul_model = Sequential([
            Dense(64, activation='relu', input_shape=(X_test.shape[1],)),
            Dropout(0.3),
            Dense(32, activation='relu'),
            Dense(1)
        ])
        
        rul_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
        
        # Split for RUL training
        X_rul_train, X_rul_test, y_rul_train, y_rul_test_split = train_test_split(
            X_test, y_rul_test, test_size=0.5, random_state=42)
        
        rul_model.fit(X_rul_train, y_rul_train, epochs=30, batch_size=32, 
                     validation_split=0.2, verbose=0)
        
        # Predictions
        rul_pred = rul_model.predict(X_rul_test, verbose=0).flatten()
        
        mae = np.mean(np.abs(rul_pred - y_rul_test_split))
        rmse = np.sqrt(np.mean((rul_pred - y_rul_test_split)**2))
        
        print(f"MAE: {mae:.2f} hours")
        print(f"RMSE: {rmse:.2f} hours")
        
        # Visualization
        plt.figure(figsize=(10, 5))
        plt.scatter(y_rul_test_split, rul_pred, alpha=0.5)
        plt.plot([0, 5000], [0, 5000], 'r--', linewidth=2)
        plt.xlabel('True RUL (hours)')
        plt.ylabel('Predicted RUL (hours)')
        plt.title('RUL Estimation: Predicted vs True')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        return rul_model
    
    def shap_analysis(self, X_test):
        X_subset = X_test[:100]

        explainer = shap.TreeExplainer(self.rf_model)
        shap_values = explainer.shap_values(X_subset)

        # Handle classifiers vs regressors
        if isinstance(shap_values, list):
            shap_values_to_plot = shap_values[1]  # or 0 depending on the class of interest
        else:
            shap_values_to_plot = shap_values

        plt.figure(figsize=(10, 6))
        shap.summary_plot(shap_values_to_plot, X_subset,
                        feature_names=self.feature_names, show=False)
        plt.tight_layout()
        plt.show()

        print("SHAP analysis complete!")

    
    def pca_visualization(self, X_test, y_test):
        """PCA visualization of feature clusters"""
        print("\nPCA Visualization...")
        
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_test)
        
        plt.figure(figsize=(10, 6))
        scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_test, 
                            cmap='RdYlGn_r', alpha=0.6, s=50)
        plt.colorbar(scatter, label='Failure Status')
        plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)')
        plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)')
        plt.title('PCA Visualization of Fault Classes')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print(f"Total variance explained: {sum(pca.explained_variance_ratio_)*100:.1f}%")
    
    def cross_dataset_validation(self, data1, data2):
        """Train on one dataset, test on another"""
        print("\nCROSS-DATASET VALIDATION")
        print("="*70)
        
        # Prepare datasets
        X1 = self.scaler.fit_transform(data1[self.feature_names])
        y1 = data1['is_failure']
        
        X2 = self.scaler.transform(data2[self.feature_names])
        y2 = data2['is_failure']
        
        # Train on dataset 1
        self.rf_model.fit(X1, y1)
        
        # Test on dataset 2
        y_pred = self.rf_model.predict(X2)
        
        acc = accuracy_score(y2, y_pred)
        f1 = f1_score(y2, y_pred)
        
        print(f"Cross-dataset Accuracy: {acc:.4f}")
        print(f"Cross-dataset F1 Score: {f1:.4f}")
        print("="*70)
        
        return acc, f1


# # Example usage
# if __name__ == "__main__":
#     print("ENHANCED MILITARY ROBOTIC SYSTEM MAINTENANCE")
#     print("="*70)
    
#     # Initialize system
#     system = MilitaryRoboticMaintenance()
    
#     # Generate data
#     print("\nGenerating synthetic data...")
#     data = system.generate_synthetic_data(n_samples=1000)
    
#     # Preprocess
#     X_train, X_test, y_train, y_test, y_rul_train, y_rul_test = system.preprocess_data(data)
    
#     # Optimize and train Random Forest
#     system.optimize_rf_with_optuna(X_train, y_train, n_trials=30)
#     system.train_optimized_rf(X_train, y_train)
    
#     # Train deep learning models
#     history_gru, history_att, X_test_seq = system.train_deep_models(
#         X_train, X_test, y_train, y_test, epochs=30)
    
#     # Comprehensive evaluation
#     results = system.evaluate_all_models(X_test, X_test_seq, y_test, n_runs=5)
    
#     # PR Curves
#     system.plot_pr_curves(X_test, X_test_seq, y_test)
    
#     # RUL Estimation
#     rul_model = system.estimate_rul(X_test, y_rul_test)
    
#     # SHAP Analysis
#     system.shap_analysis(X_test)
    
#     # PCA Visualization
#     system.pca_visualization(X_test, y_test)
    
#     # Cross-dataset validation (using two different samples)
#     data2 = system.generate_synthetic_data(n_samples=500)
#     system.cross_dataset_validation(data, data2)
    
#     print("\nALL ANALYSES COMPLETE!")